In [1]:
mitos = ["Menggunakan kacamata atau lensa kontak terlalu sering dapat membuat mata malas. Faktanya, kacamata atau lensa kontak membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.",
"Duduk terlalu dekat dengan layar komputer dapat merusak penglihatan. Meskipun dapat menyebabkan ketegangan mata, posisi duduk dekat dengan layar tidak merusak mata secara permanen.",
"Menggunakan wortel secara berlebihan akan meningkatkan penglihatan dengan cepat. Sementara wortel mengandung vitamin A yang baik untuk mata, konsumsi berlebihan tidak meningkatkan penglihatan.",
"Bersepeda dapat menyebabkan kebutaan. Ini adalah mitos; bersepeda sebenarnya adalah kegiatan yang baik untuk kesehatan secara keseluruhan, termasuk kesehatan mata.",
"Mata hanya perlu diperiksa jika ada masalah penglihatan. Tes mata secara berkala penting untuk mendeteksi masalah mata sejak dini, bahkan jika Anda tidak mengalami gejala.",
"Menggunakan tetes mata secara berlebihan dapat menyebabkan ketergantungan. Menggunakan tetes mata sesuai petunjuk dokter tidak menyebabkan ketergantungan.",
"Membaca dalam cahaya redup akan merusak mata. Ini hanya menyebabkan ketegangan mata sementara, tetapi tidak merusak mata secara permanen.",
"Menggosok mata saat gatal dapat menyebabkan penyebaran infeksi. Menggosok mata yang gatal dapat menyebabkan infeksi jika tangan Anda kotor, tetapi bukan karena tindakan menggosoknya itu sendiri.",
"Mata harus beristirahat sebentar setelah melihat layar komputer. Meskipun istirahat adalah ide yang baik, tidak ada bukti bahwa istirahat sebentar setelah melihat layar akan memperbaiki masalah penglihatan.",
"Terlalu banyak membaca dalam cahaya rendah atau membaca dengan buruk dapat merusak mata. Meskipun membaca dalam kondisi pencahayaan yang buruk dapat menyebabkan ketegangan mata sementara, itu tidak akan merusak mata secara permanen.",
]

fakta = ["Paparan sinar ultraviolet (UV) dari sinar matahari dapat menyebabkan kerusakan mata. Penggunaan kacamata hitam yang memiliki perlindungan UV dapat membantu mengurangi risiko ini.",
"Diabetes dapat mempengaruhi kesehatan mata. Kondisi ini dapat menyebabkan retinopati diabetik, yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan.",
"Perubahan penglihatan terjadi secara alami seiring bertambahnya usia. Ini termasuk presbyopia, yang menyebabkan kesulitan melihat objek dekat pada usia lanjut.",
"Merokok dapat meningkatkan risiko terjadinya katarak dan degenerasi makula, dua kondisi mata yang dapat menyebabkan kehilangan penglihatan.",
"Konsumsi makanan bergizi, seperti makanan kaya vitamin A, C, E, dan mineral seperti seng, dapat membantu menjaga kesehatan mata.",
"Mata kering adalah kondisi yang umum terjadi, terutama pada mereka yang banyak bekerja di depan layar komputer. Istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering.",
"Penggunaan komputer dan perangkat elektronik lainnya dalam jangka waktu yang lama dapat menyebabkan Computer Vision Syndrome (CVS), menyebabkan gejala seperti mata kering, sakit kepala, dan ketegangan leher.",
"Bentuk fisik mata setiap orang berbeda. Ada tiga tipe mata utama: mata datar, mata cekung, dan mata normal. Ini dapat mempengaruhi pemilihan kacamata atau lensa kontak yang tepat.",
"Kacamata hitam yang baik melindungi mata dari sinar UV dan meminimalkan paparan sinar biru dari layar komputer atau perangkat elektronik lainnya.",
"Deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius. Berkunjung ke dokter mata secara berkala dapat membantu mendeteksi dan mengatasi masalah mata sejak dini."
]

In [2]:
import nltk
import random
import string
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib

In [3]:
random.shuffle(mitos)
random.shuffle(fakta)

In [4]:
newMitos = [(here, "mitos") for here in mitos]
print(newMitos[0])
newFakta = [(here, "fakta") for here in fakta]
print(newFakta[0])

('Menggosok mata saat gatal dapat menyebabkan penyebaran infeksi. Menggosok mata yang gatal dapat menyebabkan infeksi jika tangan Anda kotor, tetapi bukan karena tindakan menggosoknya itu sendiri.', 'mitos')
('Konsumsi makanan bergizi, seperti makanan kaya vitamin A, C, E, dan mineral seperti seng, dapat membantu menjaga kesehatan mata.', 'fakta')


In [5]:
data = newMitos + newFakta
random.shuffle(data)
print(data[0:2])

[('Diabetes dapat mempengaruhi kesehatan mata. Kondisi ini dapat menyebabkan retinopati diabetik, yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan.', 'fakta'), ('Deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius. Berkunjung ke dokter mata secara berkala dapat membantu mendeteksi dan mengatasi masalah mata sejak dini.', 'fakta')]


In [6]:
"""
def preprocess_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Remove punctuation and non-alphanumeric characters
    words = [word for word in words if word.isalnum()]
    
    return " ".join(words)
"""

def clean_and_lower(text):
    # Remove non-alphanumeric characters and underscore
    cleaned_text = ''.join(char.lower() for char in text if char.isalnum() or char.isspace())
    return cleaned_text

In [7]:
# clean_and_lower(text) for text in test
hasilPre = [(clean_and_lower(words), category) for words, category in data]
print(hasilPre[0:2])

[('diabetes dapat mempengaruhi kesehatan mata kondisi ini dapat menyebabkan retinopati diabetik yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan', 'fakta'), ('deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius berkunjung ke dokter mata secara berkala dapat membantu mendeteksi dan mengatasi masalah mata sejak dini', 'fakta')]


In [8]:
train, test = train_test_split(hasilPre, test_size=0.3, random_state=42)

# Separate the features (reviews) and labels (positive/negative)
train_texts, train_labels = zip(*train)
test_texts, test_labels = zip(*test)

In [9]:
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(train_texts)

# Transform the testing data
X_test = vectorizer.transform(test_texts)

In [12]:
filenamee = 'vector.joblib'

# Save the model to the file
joblib.dump(X_train, filenamee)

['vector.joblib']

In [10]:
clf = RandomForestClassifier()

# Train the classifier
clf.fit(X_train, train_labels)

RandomForestClassifier()

In [11]:
filename = 'nlpModel.joblib'

# Save the model to the file
joblib.dump(clf, filename)

['nlpModel.joblib']

In [12]:
predictions = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("Classification Report:")
print(classification_report(test_labels, predictions))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_labels, predictions))

Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

       fakta       0.75      1.00      0.86         3
       mitos       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6

Confusion Matrix:
[[3 0]
 [1 2]]


In [17]:
review2 = "Kita harus ke dokter mata jika ada masalah dalam melihat" #mitos
review1 = "adshfajsfjkas" #fakta

# Preprocess the review texts
processed_review1 = clean_and_lower(review1)
processed_review2 = clean_and_lower(review2)

# Convert the processed review texts to feature vectors using the same TF-IDF vectorizer
review1_vector = vectorizer.transform([processed_review1])
review2_vector = vectorizer.transform([processed_review2])

# Make predictions using the trained classifier
prediction1 = clf.predict(review1_vector)[0]
prediction2 = clf.predict(review2_vector)[0]

# Print the predictions
print(f"Review 1 sentiment: {'Mitos' if prediction1 == 'mitos' else 'Fakta'}")
print(f"Review 2 sentiment: {'Mitos' if prediction2 == 'mitos' else 'Fakta'}")

Review 1 sentiment: Fakta
Review 2 sentiment: Fakta


In [14]:
print(processed_review2)

kita harus ke dokter mata jika ada masalah dalam melihat
